## 03. 함수 적용

## 참고자료
* [Python 완전정복 시리즈] 2편 : Pandas DataFrame 완전정복 : https://wikidocs.net/book/7188

In [1]:
import pandas as pd
import numpy as np

In [2]:
col = ['col1','col2','col3']
row = ['row1','row2','row3']
data = [[1,2,3],[4,5,6],[7,8,9]]
df = pd.DataFrame(data=data,index=row,columns=col)
df

,col1,col2,col3
row1,1,2,3
row2,4,5,6
row3,7,8,9


In [3]:
df.apply(np.sqrt)

,col1,col2,col3
row1,1.000000,1.414214,1.732051
row2,2.000000,2.236068,2.449490
row3,2.645751,2.828427,3.000000


In [4]:
df.apply(np.sum)

col1    12
col2    15
col3    18
dtype: int64

In [5]:
df.apply(np.prod, axis=0)

col1     28
col2     80
col3    162
dtype: int64

In [6]:
df.apply(np.prod, axis=1)

row1      6
row2    120
row3    504
dtype: int64

In [8]:
# result_type에 따른 차이

df.apply(lambda x : [1,2,3])

,col1,col2,col3
row1,1,1,1
row2,2,2,2
row3,3,3,3


In [9]:
df.apply(lambda x:[1,2,3], axis=1, result_type='expand')

,0,1,2
row1,1,2,3
row2,1,2,3
row3,1,2,3


In [10]:
df.apply(lambda x:[1,2,3], axis=1, result_type='reduce')

row1    [1, 2, 3]
row2    [1, 2, 3]
row3    [1, 2, 3]
dtype: object

In [11]:
df.apply(lambda x:[1,2,3], axis=1, result_type='broadcast')

,col1,col2,col3
row1,1,2,3
row2,1,2,3
row3,1,2,3


## 요소별

In [13]:
col = ['col1','col2','col3']
row = ['row1','row2','row3']
data = [[1,2,3],[4,5,6],[7,pd.NA,9]]
df = pd.DataFrame(data=data,index=row,columns=col)
df

,col1,col2,col3
row1,1,2,3
row2,4,5,6
row3,7,<NA>,9


In [15]:
df.applymap(lambda x : x**2, na_action='ignore')

,col1,col2,col3
row1,1,4,9
row2,16,25,36
row3,49,<NA>,81


## 함수내 함수 연속 적용

In [17]:
org_data = pd.DataFrame({'info':['삼성전자/3/70000','SK하이닉스/2/100000']})
org_data

,info
0,삼성전자/3/70000
1,SK하이닉스/2/100000


In [18]:
# code_name(data) 는 (종목명/수량/가격)형태인 문자열 data를 입력받아서 각각으로 분리하고 수량과 가격의 dtype을 int로 변경하는 함수

def code_name(data):
    result=pd.DataFrame(columns=['name','count','price']) 
    df = pd.DataFrame(list(data['info'].str.split('/'))) # '/ ' 로 구분하여 문자열을 나누어 리스트에 넣음
    result['name'] = df[0] # 여기엔 첫번째 값인 이름이 입력
    result['count']= df[1] # 여기엔 두번째 값인 수량이 입력
    result['price']= df[2] # 여기엔 세번째 값인 가격이 입력
    result = result.astype({'count':int,'price':int}) # count와 price를 int로 바꿈(기존str)
    return result

code_name(org_data)

,name,count,price
0,삼성전자,3,70000
1,SK하이닉스,2,100000


In [20]:
# value_cal(data,unit=' ')은 가격과 수량을 곱한다음에 단위로 unit arg를 붙이는 함수

def value_cal(data,unit=''):
    result = pd.DataFrame(columns=['name','value']) 
    result['name'] =data['name'] # 이름은 기존거를 가져옴
    result['value']=data['count']*data['price'] # value는 count * price를 입력함
    result = result.astype({'value':str}) # value를 str로 변경(단위를 붙이기 위함)
    result['value']=result['value']+unit # 단위를 붙임
    return(result)

input=code_name(org_data)
value_cal(input, '원')

,name,value
0,삼성전자,210000원
1,SK하이닉스,200000원


In [21]:
org_data.pipe(code_name).pipe(value_cal,'원')

,name,value
0,삼성전자,210000원
1,SK하이닉스,200000원


## 함수연속적용_축별

In [29]:
df = pd.DataFrame([[1,4,7],[2,5,8],[3,6,9]])
df

,0,1,2
0,1,4,7
1,2,5,8
2,3,6,9


In [30]:
ex1 = df.agg(np.prod)
ex1

0      6
1    120
2    504
dtype: int64

In [31]:
ex2 = df.agg('prod')
ex2

0      6
1    120
2    504
dtype: int64

In [32]:
# lambd 함수 사용시 열 명칭은 <lambda>가 된다

ex3 = df.agg([lambda x : min(x) * max(x)])
ex3

,0,1,2
<lambda>,3,24,63


In [33]:
# 사용자정의 함수 사용시 기본적으로 함수명 열 이름으로 설정

def func_sub(input):
    return max(input)-min(input)
ex4 = df.agg([func_sub,'sum'])
ex4

,0,1,2
func_sub,2,2,2
sum,6,15,24


In [34]:
# 함수명을 __name__메서드를 통해 따로 설정해주면 그 이름이 쓰임

def func_sub(input):
    return max(input)-min(input)
func_sub.__name__='내함수'
ex5 = df.agg([func_sub,'sum'])
ex5

,0,1,2
내함수,2,2,2
sum,6,15,24


In [35]:
ex6 = df.agg(['min','max','sum','prod'])
ex6

,0,1,2
min,1,4,7
max,3,6,9
sum,6,15,24
prod,6,120,504


In [36]:
ex7 = df.agg({2:'sum',0:'max',1:'min'})
ex7

2    24
0     3
1     4
dtype: int64

In [38]:
ex8 =  df.agg({0:['sum','prod'],1:['max','min'],2:'mean'}) 
ex8

,0,1,2
sum,6.0,NaN,NaN
prod,6.0,NaN,NaN
max,NaN,6.0,NaN
min,NaN,4.0,NaN
mean,NaN,NaN,8.0


## 함수연속적용_요소별

In [39]:
col = ['col1','col2','col3']
row = ['row1','row2','row3']
df = pd.DataFrame(data=[[10,40,70],[20,50,80],[30,60,90]],index=row,columns=col)
df

,col1,col2,col3
row1,10,40,70
row2,20,50,80
row3,30,60,90


In [40]:
ex1 = df.transform(np.sqrt)
ex1

,col1,col2,col3
row1,3.162278,6.324555,8.366600
row2,4.472136,7.071068,8.944272
row3,5.477226,7.745967,9.486833


In [42]:
ex2 = df.transform('sqrt')
ex2

,col1,col2,col3
row1,3.162278,6.324555,8.366600
row2,4.472136,7.071068,8.944272
row3,5.477226,7.745967,9.486833


In [43]:
ex3 = df.transform(lambda x : np.sqrt(x))
ex3

,col1,col2,col3
row1,3.162278,6.324555,8.366600
row2,4.472136,7.071068,8.944272
row3,5.477226,7.745967,9.486833


In [44]:
ex4 = df.transform(['exp','sqrt'])
ex4

col1                    col2                    col3          
               exp      sqrt           exp      sqrt           exp      sqrt
row1  2.202647e+04  3.162278  2.353853e+17  6.324555  2.515439e+30  8.366600
row2  4.851652e+08  4.472136  5.184706e+21  7.071068  5.540622e+34  8.944272
row3  1.068647e+13  5.477226  1.142007e+26  7.745967  1.220403e+39  9.486833

In [45]:
ex5 = df.transform({'col2':'exp', 'col1':'sqrt'})
ex5

,col2,col1
row1,2.353853e+17,3.162278
row2,5.184706e+21,4.472136
row3,1.142007e+26,5.477226


## 문자열 형식의 계산식 적용

In [46]:
data = [[1,2,3],[4,5,6],[7,8,9]]
col = ['col1','col2','col3']
row = ['row1','row2','row3']
df = pd.DataFrame(data = data, index = row, columns= col)
df

,col1,col2,col3
row1,1,2,3
row2,4,5,6
row3,7,8,9


In [47]:
df.eval('col4=col1*col2-col3')

,col1,col2,col3,col4
row1,1,2,3,-1
row2,4,5,6,14
row3,7,8,9,47


In [48]:
df

,col1,col2,col3
row1,1,2,3
row2,4,5,6
row3,7,8,9


In [49]:
df.eval('col4=col1*col2-col3', inplace=True)

In [50]:
df

,col1,col2,col3,col4
row1,1,2,3,-1
row2,4,5,6,14
row3,7,8,9,47
